# Application Programming Interfaces (APIs)
Watch [this video](https://www.youtube.com/watch?v=bqg6lDsD0-I&ab_channel=DominiquePaul) that I created for a detailed explanation.

To make API request in Python we use the requests package (again, so convenient that somebody already implemented this for us!). In this case we will make a request to a weather API to get the current weather for Zurich.

As explained in the video, we need to know where we want to send the request (i.e. the endpoint) and we should know what we can send there (parameters). In our case we will also need to have an API key ready to access the API.


## Using the OpenWeather API
You can find out more about the API we are using and create an account for free here:
https://openweathermap.org/

You can read up the documentation of the API we are calling here:
https://openweathermap.org/current


In [1]:
# you might need to run this
#!pip install requests

import requests

We can read up on the [documentation](https://requests.readthedocs.io/en/master/user/quickstart/) to understand how to make a API request using the requests package.

We will first define all of the information we need to make the request.

In [2]:
# fast and sloppy way to enter an api key 
WEATHER_API_KEY = "enter-your-key-here"

# Safe way to enter an api key: Load it from a file where you keep all your credentials and 
# that you don't upload or share anywhere
from secrets_file import WEATHER_API_KEY



endpoint = "http://api.openweathermap.org/data/2.5/weather"

parameters = {"q": "St. Gallen",
              "appid": WEATHER_API_KEY}

Now we can make the API call

In [3]:
r = requests.get(endpoint, params=parameters)
r

<Response [200]>

Hmm, this isn't very useful. Lets inspect the object that the package returns to us:

In [4]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Of these options it would presumably be worthwhile checking out: url, request, status_code, text and text

## Other attributes of the response object
The query sent (notice how the parameters got added to the url)

In [5]:
r.url

'http://api.openweathermap.org/data/2.5/weather?q=St.+Gallen&appid=your-api-key-here'

Also not what were looking for

In [6]:
r.request

<PreparedRequest [GET]>

The status code tells us whether our request was successful. If you run into errors, you can check out all status codes here:
https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

In [7]:
r.status_code

200

This is what we were looking for. But this is text only and difficult to read out information if the response is longer. so what we normally use is .json() which gives us the response in a python dictionary format

In [8]:
r.text

'{"coord":{"lon":9.37,"lat":47.42},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"base":"stations","main":{"temp":280.83,"feels_like":279.49,"temp_min":279.26,"temp_max":282.15,"pressure":1009,"humidity":87},"visibility":10000,"wind":{"speed":0.5,"deg":0},"clouds":{"all":90},"dt":1601247601,"sys":{"type":1,"id":6944,"country":"CH","sunrise":1601270258,"sunset":1601312904},"timezone":7200,"id":2658822,"name":"St. Gallen","cod":200}'

This is what we want!

In [11]:
r.json()

{'coord': {'lon': 9.37, 'lat': 47.42},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 280.83,
  'feels_like': 279.49,
  'temp_min': 279.26,
  'temp_max': 282.15,
  'pressure': 1009,
  'humidity': 87},
 'visibility': 10000,
 'wind': {'speed': 0.5, 'deg': 0},
 'clouds': {'all': 90},
 'dt': 1601247601,
 'sys': {'type': 1,
  'id': 6944,
  'country': 'CH',
  'sunrise': 1601270258,
  'sunset': 1601312904},
 'timezone': 7200,
 'id': 2658822,
 'name': 'St. Gallen',
 'cod': 200}

In [12]:
data = r.json()
data

{'coord': {'lon': 9.37, 'lat': 47.42},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 280.83,
  'feels_like': 279.49,
  'temp_min': 279.26,
  'temp_max': 282.15,
  'pressure': 1009,
  'humidity': 87},
 'visibility': 10000,
 'wind': {'speed': 0.5, 'deg': 0},
 'clouds': {'all': 90},
 'dt': 1601247601,
 'sys': {'type': 1,
  'id': 6944,
  'country': 'CH',
  'sunrise': 1601270258,
  'sunset': 1601312904},
 'timezone': 7200,
 'id': 2658822,
 'name': 'St. Gallen',
 'cod': 200}

Great! That looks like a dictionary and we luckily are quite familiar with dictionaries. 

We could also have googled "how to read requests object python" which would either get us to stackoverflow or to the website of W3 schools with [this article](how to read requests object python). There we could have checked in more detail what the good options are.

Now lets extract the temperature.

In [13]:
temperature = data["main"]["temp"]
temperature

280.83

Thats in kelvin! We would rather have it in celsius, so we read through the documentation to see if we can possibly change that.

And we're lucky, we can just add the parameter "units" to our request:

In [14]:
parameters["units"] = "metric"
r = requests.get(endpoint, params=parameters)
data = r.json()

temperature = data["main"]["temp"]
temperature

print(f"Right now the weather in St. Gallen is {temperature} degrees celsius")

Right now the weather in St. Gallen is 7.49 degrees celsius


## Minitask 1:
Turn the code we just wrote into a function that takes the argument "location" and returns the temperature in celsius for the location

In [15]:
# your code

# Using the currency exchange API
Get your API key and read the documentation here: https://rapidapi.com/fyhao/api/currency-exchange

We again load our API key and the store the endpoint as a variable

In [17]:
# Never save API keys in your code. You can get your own API key at
# https://rapidapi.com/fyhao/api/currency-exchange
from secrets_file import FX_API_KEY


endpoint = "https://currency-exchange.p.rapidapi.com/exchange"

This API is a bit different. It does not only ask for parameters, but also for headers.

Headers include important information to be sent with the message but which do not form part of the request itself. In our case, they contain information about the resource to be fetched and 'who' is asking for it (API key). API keys can asked for as a parameter or as a header, but generally headers are the safer option (See [this stackoverflow question](https://stackoverflow.com/questions/5517281/place-api-key-in-headers-or-url))

In [18]:
headers = {
    'x-rapidapi-host': "currency-exchange.p.rapidapi.com",
    'x-rapidapi-key': FX_API_KEY
    }

We also need parameters here for some generic information. The parameters contain information about the query itself (not the user who is making it): what information are we asking for?

In [19]:
parameters = {"q":"1.0",
              "from":"EUR",
              "to":"USD"}

In [20]:
r = requests.get(endpoint, headers=headers, params=parameters)
fx_rate = r.json()
fx_rate

1.162534

In [21]:
print(f"The exchange rate of the euro to the dollar is: {fx_rate}")


The exchange rate of the euro to the dollar is: 1.162534


## Minitask 2
Again, wrap this into a function

# Your turn: Querying the Nasa API
Its your turn! Have a look at Nasa's APIs which are online for free. Download your API key from here: https://api.nasa.gov

Try to do the following, but careful you might have to do some additional work to understand how to display images

1. Make a request to the Astronomy picture of the day api and inspect the response
2. Try to display the image from the image url provided
3. Retrieve and display the 'image of the day' from the 1s of October 2019
4. Retrieve an image of a date of your choice and save it to your local directory